In [8]:
!git clone https://github.com/roihezkiyahu/CWGAN-GP.git

Cloning into 'CWGAN-GP'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 56 (delta 27), reused 16 (delta 8), pack-reused 0
Unpacking objects: 100% (56/56), 4.70 MiB | 6.24 MiB/s, done.


In [9]:
!pip install lightning

In [10]:
%cd CWGAN-GP

/kaggle/working/CWGAN-GP/CWGAN-GP


In [11]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from Generator import GeneratorMNIST
from Discriminator import DiscriminatorMNIST
from GAN import GAN
import lightning as pl
from PIL import Image
from IPython.display import Image as DisplayImage, display
import os
import re
import numpy as np
import matplotlib.pyplot as plt

In [12]:
batch_size = 64
learning_rate_wgan = 2e-4
num_epochs = 150
lambda_gp = 10
latent_dim = 128
dim = 128
n_critics = 1
step_lr_gamma = 0.975
betas = (0.5, 0.999)
acc_device = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(acc_device)
dataset = "fashion_mnist"

In [15]:
def load_dataset(dataset, pad=2, affine_rotation=5, affine_translation=(0.1, 0.1), affine_scale=(0.95, 1.05),
                 normalize_mean=(0.5), normalize_std=(0.5)):
    if dataset == "CIFAR10":
        transform = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomAffine(affine_rotation, affine_translation, scale=affine_scale),
            transforms.ToTensor(),
            transforms.Normalize(normalize_mean, normalize_std)
        ])
        train_set = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
        test_set = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    elif dataset == "mnist" or dataset == "fashion_mnist":
        transform = transforms.Compose([
            transforms.Pad(pad),
            transforms.ToTensor(),
            transforms.RandomAffine(affine_rotation, affine_translation, scale=affine_scale),
            transforms.Normalize(normalize_mean, normalize_std)
        ])
        if dataset == "mnist":
            train_set = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
            test_set = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
        else:
            train_set = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
            test_set = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
    else:
        raise ValueError("Invalid dataset string. Supported options: 'CIFAR10', 'mnist', 'fashion_mnist'")
    
    class_names = train_set.classes
    
    full_dataset = torch.utils.data.ConcatDataset([train_set, test_set])
    return full_dataset, class_names

if dataset == "CIFAR10":
    normalize_mean = (0.5, 0.5, 0.5)
    normalize_std = (0.5, 0.5, 0.5)
    img_size = (32, 32, 3)
else:
    normalize_mean = (0.5)
    normalize_std = (0.5)
    img_size = (32, 32, 1)
full_dataset, class_names = load_dataset(dataset, normalize_mean=normalize_mean, normalize_std=normalize_std)

train_loader = torch.utils.data.DataLoader(full_dataset, batch_size=batch_size,
                                          shuffle=True, num_workers=2, pin_memory=True)

Files already downloaded and verified
Files already downloaded and verified


In [16]:
# Configurations and other code from your script...

# CWGAN-GP
generator_wgan_mnist = GeneratorMNIST(img_size, latent_dim, dim, batch_norm=True)
discriminator_wgan_mnist = DiscriminatorMNIST(img_size, dim, batch_norm=False)
wgan_mnist = GAN(generator_wgan_mnist, discriminator_wgan_mnist, True, lambda_gp, device, learning_rate_wgan,
                 betas, n_critics, step_lr_gamma=step_lr_gamma, class_names=class_names)


# Training
trainer = pl.Trainer(accelerator=acc_device, max_epochs=num_epochs)
trainer.fit(wgan_mnist, train_loader)

INFO: GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO: 
  | Name               | Type               | Params
----------------------------------------------------------
0 | generator          | GeneratorMNIST     | 3.9 M 
1 | discriminator      | DiscriminatorMNIST | 1.7 M 
2 | generator_loss     | BCELoss            | 0     
3 | discriminator_loss | BCELoss            | 0     
----------------------------------------------------------
5.5 M     Trainable params
0         Non-trainable params
5.5 M     Total params
22.174    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]